In [22]:
import pandas as pd
import numpy as np
import os
import time
from tqdm import tqdm

In [35]:
column_names = ['ms','ankle_horiz_fw','ankle_vert', 'ankle_horiz_lat','thigh_horiz_fw','thigh_vert', 'thigh_horiz_lat','trunk_horiz_fw','trunk_vert', 'trunk_horiz_lat','annotation']
datadir = 'dataset_fog_release/dataset'
outputdir = 'sliding_window_dataset/'

subject_files = {'s1': ['S01R01.txt','S01R02.txt'],'s2':['S02R01.txt','S02R02.txt'],'s3':['S03R01.txt','S03R02.txt','S03R03.txt'],
                 's4':['S04R01.txt'],'s5':['S05R01.txt','S05R02.txt'],'s6':['S06R01.txt','S06R02.txt'],'s7':['S07R01.txt','S07R02.txt'],
                 's8': ['S08R01.txt'],'s9': ['S09R01.txt'],'s10':['S10R01.txt']}
subj_file = subject_files["s1"][0]
filename = f'{datadir}/{subj_file}'
df = pd.read_csv(filename,sep=' ', names = column_names)

In [36]:
def extract_experiment_windows(annotations):
    '''Extracts the indices where an active experiment occurs for slicing time series into valid chunks
    annotations : [int]
        an array of annotations where 0 is inactive, 1 is active experiment, and 2 is gait freeze. Interested in the slices
        of this array that are not equal to 0
    '''
    start = None
    stop = None
    windows = []
    for i in range(len(annotations)):
        if annotations[i] != 0 and not start:
            start = i
        if annotations[i] == 0 and start:
            end = i - 1
            windows.append((start,end))
            start = None
            end = None
    return windows

In [37]:

def create_and_save_sliding_windows(subj_data_frame,filename, outputdir, window_size = 1000, columns=['ankle_vert','annotation']):
    '''given a subject's dataframe, this will use extract_experiment_windows to find the time intervals where there is 
    an active experiment, and then return a numpy array of "sliding windows".
    By default, the only column pulled will be vertical ankle acceleration
    i.e. 
    if I have a time series [1,2,3,4,5,6], and a window size of 3, this function returns ([[1],[2],[3]], [4]), ([[2],[3],[4]],[5]), etc. 
    We want something of the form:
    [[ankle_acc1,...,annotation1] ,[ankle_acc2,...,annotation2],...,[ankle_acc_winsize-1,...,annotation_winsize-1]] , [annotation_winsize]
    '''
    experiment_intervals = extract_experiment_windows(subj_data_frame['annotation'])
    df_as_np = subj_data_frame.to_numpy()

    experiment_intervals = extract_experiment_windows(df['annotation'])
    numpy_df = df[columns].to_numpy()
    numpy_df_intervals = []
    X = []
    y = []
    start = time.time()
    for interval in experiment_intervals:
        numpy_df_intervals.append(numpy_df[interval[0]:interval[1]])
    for experiment,n in zip(numpy_df_intervals, range(len(numpy_df_intervals))):
        for i in tqdm(range(len(experiment) - window_size)):
            row = [a for a in experiment[i:i+window_size]]
            X.append(row)
            label = experiment[i+window_size][3]
            y.append(0 if label == 1 else 1)
        np.save(f'{outputdir}{filename[0:6]}experiment{n}_X.npy',X)
        np.save(f'{outputdir}{filename[0:6]}experiment{n}_y.npy',y)

    end = time.time()
    print(end-start)
create_and_save_sliding_windows(df, subj_file, outputdir ,columns = ['ankle_horiz_fw','ankle_vert', 'ankle_horiz_lat','annotation'])

100%|██████████████████████████████████████████████████████████████████████████| 72600/72600 [01:10<00:00, 1022.88it/s]


232.2439603805542


In [38]:
X = np.load('sliding_window_dataset/S01R01experiment1_X.npy')
y = np.load('sliding_window_dataset/S01R01experiment1_y.npy')

In [40]:
print(X.shape)
print(y.shape)
print(X[0])
print(np.where(y==1)[0])



(90800, 1000, 4)
(90800,)
[[ -30  990  326    1]
 [ -30 1000  356    1]
 [ -20  990  336    1]
 ...
 [-131  970  366    1]
 [-111  960  346    1]
 [ -90 1000  366    1]]
[19745 19746 19747 ... 85854 85855 85856]


In [41]:
column_names = ['ms','ankle_horiz_fw','ankle_vert', 'ankle_horiz_lat','thigh_horiz_fw','thigh_vert', 'thigh_horiz_lat','trunk_horiz_fw','trunk_vert', 'trunk_horiz_lat','annotation']
datadir = 'dataset_fog_release/dataset'
outputdir = 'sliding_window_dataset/'

subject_files = {'s1': ['S01R01.txt','S01R02.txt'],'s2':['S02R01.txt','S02R02.txt'],'s3':['S03R01.txt','S03R02.txt','S03R03.txt'],
                 's4':['S04R01.txt'],'s5':['S05R01.txt','S05R02.txt'],'s6':['S06R01.txt','S06R02.txt'],'s7':['S07R01.txt','S07R02.txt'],
                 's8': ['S08R01.txt'],'s9': ['S09R01.txt'],'s10':['S10R01.txt']}
for subj in subject_files.keys():
    for fname in subject_files[subj]:
        path_to_file = f'{datadir}/{fname}'
        df = pd.read_csv(path_to_file,sep=' ', names = column_names)
        create_and_save_sliding_windows(df, fname, outputdir ,columns = ['ankle_horiz_fw','ankle_vert', 'ankle_horiz_lat','annotation'])

100%|██████████████████████████████████████████████████████████████████████████| 72600/72600 [00:49<00:00, 1462.63it/s]


195.61806964874268


100%|██████████████████████████████████████████████████████████████████████████| 27800/27800 [00:08<00:00, 3166.64it/s]


22.216312885284424


100%|██████████████████████████████████████████████████████████████████████████| 24600/24600 [00:06<00:00, 3835.09it/s]


...

76.1907594203949


100%|████████████████████████████████████████████████████████████████████████████| 8600/8600 [00:04<00:00, 2121.36it/s]


629.2478468418121


100%|███████████████████████████████████████████████████████████████████████████| 90520/90520 [03:42<00:00, 406.52it/s]


704.4332501888275
